# CLAUDIO RICCI

## Packages

In [2]:
import networkx as nx # For graphs
import pickle # For data parsing
from networkx.algorithms.approximation import greedy_tsp # For approx TSP
import numpy as np

import torch
from torch.utils.data import DataLoader, random_split, Dataset
import torch.nn as nn
from torch.nn import Transformer
from torch import Tensor

import warnings
from timeit import default_timer as timer
import math

## Helper functions

In [3]:
def tour_length(G, tour):
    """
    Compute the length of a tour. A tour is a list having elments 0 and -1 equal
    """
    assert tour[0] == tour[-1], "Not valid tour"
    estimated = 0
    for i in range(n):
        estimated += G[tour[i]][tour[i + 1]]['weight']
    return estimated

def greedy_algorithm(G):
    """
    Run the value of the greedy approximation algorithm on graph G
    """
    return tour_length(G, greedy_tsp(G, weight='weight'))

def random_tour(G, seed = 42):
    """
    Return the value of a random tour
    """
    np.random.seed(seed)
    n = G.number_of_nodes()
    tour = [0]
    for i in range(1, n):
        next_node = np.random.choice([j for j in range(n) if j not in tour])
        tour.append(next_node)
    tour.append(0)

def transformer_tsp(G, model, DEVICE = 'cpu'):
    """
    Evaluate your (trained) model on G
    """
    # Set the model in evaluation mode
    model.eval()

    # Note: number of edges is constant ed equal to n(n-1)/2
    n = G.number_of_nodes()
    
    # Get node coordinates
    attr = nx.get_node_attributes(G, 'pos')
    x = []
    for i in range(n):
        x.append(torch.tensor(attr[i], dtype=torch.float32))

    # From list of tensors to tensor 2d
    x = torch.stack(x)    

    tour = [0]
    y = torch.tensor(tour, dtype=torch.long)
    x = x.to(DEVICE).unsqueeze(0)
    y = y.to(DEVICE).unsqueeze(0)
    
    # Predict the next node
    out = transformer_model(x, y)
    
    # Loop until the tour is complete
    while len(tour) < n:
        _, idx = torch.topk(out, n, dim=2)
        for i in range(n):
            # Check if the node is already in the tour
            if idx[0, 0, i] not in tour:
                tour.append(idx[0, 0, i])
                break
        y = torch.tensor(tour)
        y = y.to(DEVICE).unsqueeze(0)
        out = transformer_model(x, y)
    
    tour = [int(i) for i in tour] + [0] # Append the starting node (that is hard-coded to 0)
    return tour_length(G, tour)



def gap(G, model = None, model_GA = None, random_seed = 42, device = 'cpu'):
    """
    Compute the gap between the optimal solution on graph G and all the analyzed methods
    """

        
    # Optimal value (hard-coded in the graph)
    TSP = sum([G[i][j]['weight']*G[i][j]['tour'] for (i, j) in G.edges()]) # Optimal

    # Gaps dictionary
    gaps = {'greedy' : 0, 'random' : 0, 'transformer_tsp': 0, 'transformer_tsp_acc_grad': 0}
    gaps['greedy'] = 100* (greedy_algorithm(G) -  TSP) / TSP
    gaps['random'] = 100 * (random_tour(G, random_seed) - TSP) / TSP
    if model is not None:
        gaps['transformer_tsp'] = 100 * (transformer_tsp(G, model, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp'] = float('inf') # In case you just train with GA
        
    if model_GA is not None:
        gaps['transformer_tsp_acc_grad'] = 100 * (transformer_tsp(G, model_GA, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp_acc_grad'] = float('inf') # In case you just train without GA
    return gaps    
    

## Dataset & Dataloader

In [4]:
# Load the dummy dataset, get a single data item and explain its Python type
with open('/kaggle/input/tspinstances/dummy_20_DLL_ass4.pkl', 'rb') as file:
    dummy = pickle.load(file)

print(type(dummy)) # list
print('Type of a element of the dataset: ', type(dummy[0]))  # The type of the first object -> tuple
print('Type of the first item of the tuple: ', type(dummy[0][0]))  # The type of the first item of a tuple -> Graph
print('Type of the second item of the tuple: ', type(dummy[0][1]))  # The type of the first item of a tuple -> list
print(dummy[0])

<class 'list'>
Type of a element of the dataset:  <class 'tuple'>
Type of the first item of the tuple:  <class 'networkx.classes.graph.Graph'>
Type of the second item of the tuple:  <class 'list'>
(<networkx.classes.graph.Graph object at 0x7851001e5420>, [0, 3, 14, 2, 9, 6, 19, 13, 12, 16, 7, 18, 8, 17, 5, 11, 10, 15, 1, 4, 0])


In [5]:
dummy[0][0].edges

EdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (7, 8), (7, 9), (7, 10),

In [6]:
# Describe the edge attributes tour and weight

# Extract the graph and tour
graph = dummy[0][0]  # The networkx Graph object
tour = dummy[0][1]   # The tour as a list of nodes

# Inspect edges with attributes
print("Edges and attributes:")
for u, v, data in graph.edges(data=True):
    print(f"  Edge ({u}, {v}):")
    print('     Weight: ', {data.get('weight', 'Not found')})
    print('     Tour: ',{data.get('tour', 'Not found')})

Edges and attributes:
  Edge (0, 1):
     Weight:  {0.4287846201876535}
     Tour:  {0}
  Edge (0, 2):
     Weight:  {0.20417626260418414}
     Tour:  {0}
  Edge (0, 3):
     Weight:  {0.08154537102129383}
     Tour:  {1}
  Edge (0, 4):
     Weight:  {0.08031174728403137}
     Tour:  {1}
  Edge (0, 5):
     Weight:  {0.5612080164024046}
     Tour:  {0}
  Edge (0, 6):
     Weight:  {0.2295012043541082}
     Tour:  {0}
  Edge (0, 7):
     Weight:  {0.514746498076055}
     Tour:  {0}
  Edge (0, 8):
     Weight:  {0.5266704818162102}
     Tour:  {0}
  Edge (0, 9):
     Weight:  {0.15283720045858457}
     Tour:  {0}
  Edge (0, 10):
     Weight:  {0.3556451198878696}
     Tour:  {0}
  Edge (0, 11):
     Weight:  {0.24046942531001395}
     Tour:  {0}
  Edge (0, 12):
     Weight:  {0.3189080728571583}
     Tour:  {0}
  Edge (0, 13):
     Weight:  {0.49621109228471955}
     Tour:  {0}
  Edge (0, 14):
     Weight:  {0.1765729477183786}
     Tour:  {0}
  Edge (0, 15):
     Weight:  {0.47016083591

In [7]:
# Inspect the node attribute pos
print("\nNodes and attributes:")
for node, data in graph.nodes(data=True):
    print(f"  Node {node}:")
    print('     Position: ',{data.get('pos', 'Not found')})


Nodes and attributes:
  Node 0:
     Position:  {(0.6049077053425551, 0.5748590937018008)}
  Node 1:
     Position:  {(0.38474987528197846, 0.9428085200806016)}
  Node 2:
     Position:  {(0.6102491981278754, 0.3707527129445174)}
  Node 3:
     Position:  {(0.5497610140601452, 0.514788385568776)}
  Node 4:
     Position:  {(0.5941533303116413, 0.6544475361385552)}
  Node 5:
     Position:  {(0.06187381797691738, 0.433195284467101)}
  Node 6:
     Position:  {(0.7475717305758963, 0.3950876312718402)}
  Node 7:
     Position:  {(0.6548530739834322, 0.06254140180263457)}
  Node 8:
     Position:  {(0.2210796367473482, 0.2142238067774731)}
  Node 9:
     Position:  {(0.6696714621150585, 0.4364218673039507)}
  Node 10:
     Position:  {(0.3206284506117195, 0.7885615893113229)}
  Node 11:
     Position:  {(0.37760295594024584, 0.4963855605324464)}
  Node 12:
     Position:  {(0.8300002368321365, 0.3489482457969727)}
  Node 13:
     Position:  {(0.9983722301898076, 0.2725163812162502)}
  Nod

In [8]:
# # Analyze the tour
# print("\nTour edges:")
# tour_edges = [(tour[i], tour[i+1]) for i in range(len(tour) - 1)]
# for u, v in tour_edges:
#     if graph.has_edge(u, v):
#         print(f"   Edge ({u}, {v}) exists with weight {graph[u][v].get('weight', 'Not found')}.")
#     else:
#         print(f"   Edge ({u}, {v}) does not exist in the graph.")

In [9]:
torch.manual_seed(0)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

In [10]:
# Implement a dataset class. Focus on the getitem method to return:
# – X: A tensor of node coordinates with size 20 × 2.
# – y: A tour starting from 0 and ending with 0.

class GraphDataset(Dataset):
    def __init__(self, data):
        """
        A list of tuples where each tuple contains:
        - A networkx.Graph object
        - A tour (list of node indices)
        """
        self.data = data

    def __len__(self):
        """
        Number of instances in the dataset
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns:
        - X: A tensor of node coordinates with size 20 × 2.
        - y: A tensor representing the tour, starting and ending at 0.
        """
        # Extract the graph and tour
        graph, tour = self.data[idx]

        # Get node positions as a 2D array
        pos = nx.get_node_attributes(graph, 'pos')  # Dictionary {node: (x, y)}
        if not pos:
            raise ValueError(f"Graph at index {idx} is missing node positions ('pos').")

        # Ensure nodes are sorted by their index (important for consistent tensor order)
        sorted_positions = [pos[node] for node in sorted(graph.nodes())]
        
        # Convert positions to a tensor of shape (20, 2)
        X = torch.tensor(sorted_positions, dtype=torch.float32)

        # Convert the tour to a tensor
        y = torch.tensor(tour, dtype=torch.long)

        return X, y

In [11]:
# Create Dataset objects for training, validation, and testing, along with their respective Dataloader
dataset = GraphDataset(dummy)
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1])

batch_size = 32

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Model

In [12]:
class PositionalEncoding(nn.Module):
    def __init__(self,emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()

        pos_embedding = torch.zeros((maxlen, emb_size))
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)   
        
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TSPTransformer(nn.Module):
    def __init__(self, n, num_encoder, num_decoder, de, dd, n_head, dropout):
        super(TSPTransformer, self).__init__()
        # Encoder
        self.linear1 = nn.Linear(2, de)
        encoder_layer = nn.TransformerEncoderLayer(d_model=de, nhead=n_head) # d_model (int) – the number of expected features in the input (required).
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder) # stack of n encoder layers
        self.linear2 = nn.Linear(de, dd)
        
        # Decoder
        self.embedding = nn.Embedding(n, dd)
        self.posEncoding = PositionalEncoding(dd, dropout)
        decoder_layer = nn.TransformerDecoderLayer(d_model=dd, nhead=n_head)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder)
        self.ffnn = nn.Linear(dd, n)

    def forward(self,
            src: Tensor,
            trg: Tensor,
            src_mask: Tensor,
            tgt_mask: Tensor,
            src_padding_mask: Tensor,
            tgt_padding_mask: Tensor,
            memory_key_padding_mask: Tensor):
        # Encoding
        src = self.linear1(src)
        src = self.encoder(src, mask=src_mask, src_key_padding_mask=src_padding_mask)
        src = self.linear2(src)
        
        # Decoding
        trg = self.embedding(trg)
        trg = self.posEncoding(trg)
        output = self.decoder(trg, src, tgt_mask=tgt_mask, memory_mask=None,
                              tgt_key_padding_mask=tgt_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
        output = self.ffnn(output)
        return output

In [13]:
def generate_square_subsequent_mask(sz):
    ## Decoder mask
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[1]  
    tgt_seq_len = tgt.shape[1]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    # Corrected padding mask dimensions (batch_size, sequence_length)
    src_padding_mask = torch.zeros((src.shape[0], src_seq_len), device=DEVICE).bool()  # Corrected
    tgt_padding_mask = torch.zeros((tgt.shape[0], tgt_seq_len), device=DEVICE).bool()  # Corrected

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

$n$ = num di nodes -> Each TSP instance has exactly 20 nodes, and the input consists of their 2D coordinates.

$de$ = size of the internal rapresentation of the input coordinates
       2 feature per node -> small values like 16, 32, 64

$dd$ = size of intermediate representation
     y is a sequence of discrete node indeces, typical equal to de but also higher

$Ne$ = num encoeer layers -> 2,4,6

$Nd$ = num decoder layers -> equal to Ne but also higher

In [14]:
n = 20
n_enc = 4
n_dec = 4
de = 32
dd = 64
N_HEAD = 8
DROPOUT = 0.1

TSPmodel = TSPTransformer(n, n_enc, n_dec, de, dd, N_HEAD, DROPOUT).to(DEVICE)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Training

In [15]:
# Function for training a single epoch
def train_epoch(model, optimizer, trainloader, loss_fn):
    model.train()
    losses = 0

    for src, tgt in trainloader:
        src = src.to(DEVICE)  # Node coordinates (input to the encoder)
        tgt = tgt.to(DEVICE)  # Tour indices (target sequence for the decoder)

        tgt_input = tgt[:, :-1]  # Input to the decoder (shifted by one token)
        tgt_out = tgt[:, 1:]  # Target for loss computation (shifted by one token)

        optimizer.zero_grad()

        # Generate masks for attention
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # Forward pass
        output = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update weights
        optimizer.step()
        losses += loss.item()

    return losses / len(trainloader)

In [16]:
# Function for evaluation
def evaluate(model, valloader, loss_fn):
    model.eval()
    losses = 0

    with torch.no_grad():
        for src, tgt in valloader:
            src = src.to(DEVICE)  # Node coordinates (input to the encoder)
            tgt = tgt.to(DEVICE)  # Tour indices (target sequence for the decoder)
          
            tgt_input = tgt[:, :-1]  # Input to the decoder (shifted by one token)
            tgt_out = tgt[:, 1:]  # Target for loss computation (shifted by one token)

            # Generate masks for attention
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

            # Forward pass
            output = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

            loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))
            losses += loss.item()

    avg_loss = losses / len(valloader)
    return avg_loss

In [17]:
warnings.filterwarnings("ignore")

# hyperparameters
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(TSPmodel.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.01)

# Training configuration
NUM_EPOCHS = 10
training_losses = []
val_losses = []

# Training loop
for epoch in range(1, NUM_EPOCHS + 1):
    start_time = timer()
    
    # Train for one epoch
    train_loss = train_epoch(TSPmodel, optimizer, trainloader, loss_fn)    
    
    # Evaluate on validation data
    val_loss = evaluate(TSPmodel, valloader, loss_fn)
    end_time = timer()

    # Log losses
    training_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f"Epoch: {epoch}")
    print(f"Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, Perplexity: {perplexity:.3f}")
    print(f"Epoch time = {(end_time - start_time):.3f}s")

RuntimeError: The shape of the 2D attn_mask is torch.Size([20, 20]), but should be (32, 32).

In [ ]:
for src, tgt in valloader:
    src = src.to(DEVICE)  # Node coordinates (input to the encoder)
    tgt = tgt.to(DEVICE)  # Tour indices (target sequence for the decoder)
            
    print(f"src shape: {src.shape}")  # Should be (batch_size, seq_len, embedding_dim)
    print(f"tgt shape: {tgt.shape}")  # Should be (batch_size, seq_len, embedding_dim)

In [ ]:
tgt_input = tgt[:, :-1]  # Input to the decoder (shifted by one token)
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
print(f"src_mask shape: {src_mask.shape}")
print(f"src_padding_mask shape: {src_padding_mask.shape}")

src_mask = src_mask.repeat(batch_size, 1, 1)  # Repeat the mask for the batch size

### Training WITHOUT gradient accumulation

### Training WITH gradient accumulation

## Testing